<a href="https://colab.research.google.com/github/carighi/al_ml_workshop/blob/main/Outlier_Identification_and_Removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Outlier Identification and Removal**

In this tutorial, you will learn:

* That an outlier is an unlikely observation in a dataset and may have one of many causes.
* How to use simple univariate statistics like standard deviation and interquartile range to
identify and remove outliers from a data sample.
* How to use an outlier detection model to identify and remove rows from a training dataset
in order to lift predictive modeling performance.

Adapted from Jason Brownlee. 2020. [Data Preparation for Machine Learning](https://machinelearningmastery.com/data-preparation-for-machine-learning/).

Code explanations are done with aid of CS50.ai.

#Outlier Identification and Removal
##What are Outliers?

An outlier is an observation that is unlike the other observations. They are rare, distinct, or do
not fit in some way.

We will generally define outliers as samples that are exceptionally far from the
mainstream of the data.

Outliers can have many causes, such as:

* Measurement or input error.

* Data corruption.

* True outlier observation.

There is no precise way to define and identify outliers in general because of the specifics of
each dataset. Instead, you, or a domain expert, must interpret the raw observations and decide
whether a value is an outlier or not.

##Remove outliers using Standard Deviation method

We will generate a set of random numbers and calculate the mean and standard deviation.
The code below does the following:

    It imports necessary functions from the numpy library.
    It sets a seed for the random number generator. This is done to ensure that the same set of random numbers is generated each time the code is run.
    It generates 10,000 random numbers that follow a normal (Gaussian) distribution. The randn(10000) function generates these numbers with a mean of 0 and a standard deviation of 1. Multiplying by 5 and adding 50 shifts this distribution to have a mean of 50 and a standard deviation of 5.
    It calculates the mean and standard deviation of these numbers using the mean(data) and std(data) functions.
    It prints the mean and standard deviation of the generated data. The %.3f is a placeholder for a floating-point number, with 3 digits after the decimal point. The % operator is then used to insert the mean and standard deviation into these placeholders.


In [ ]:
# identifiy outliers using Standard Deviation method
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))

In [ ]:
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# define outliers. Explain code below
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off

In [ ]:
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))

In [ ]:
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

##Remove outliers using Interquartile Range method

In [ ]:
# identify outliers with interquartile range
from numpy.random import seed
from numpy.random import randn
from numpy import percentile
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50

In [ ]:
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

In [ ]:
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off

In [ ]:
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))

In [ ]:
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

##Remove outliers using Automatic Outlier Detection method

A simple approach to identifying outliers is to locate those examples that are far from the
other examples in the multi-dimensional feature space. This can work well for feature spaces
with low dimensionality (few features), although it can become less reliable as the number of
features is increased, referred to as the **curse of dimensionality**. The local outlier factor, or
LOF for short, is a technique that attempts to harness the idea of nearest neighbors for outlier
detection. Each example is assigned a scoring of how isolated or how likely it is to be outliers
based on the size of its local neighborhood. Those examples with the largest score are more
likely to be outliers.

##Diabetes Dataset
The dataset classifies patient data as
either an onset of diabetes within five years or not. There are 768 examples and eight input variables. It is a binary classification problem.

You can learn more about the dataset here:

* Diabetes Dataset File ([pima-indians-diabetes.csv](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv))
* Diabetes Dataset Details ([pima-indians-diabetes.names](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names))

###Download Diabetes data files

In [ ]:
!wget "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv" -O pima-indians-diabetes.csv
!wget "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names" -O pima-indians-diabetes.names
!head pima-indians-diabetes.csv

The following code does the following steps:

    It imports necessary functions from the pandas and scikit-learn libraries.
    It loads the dataset from a CSV file using pandas' read_csv function.
    It converts the DataFrame to a numpy array using the values attribute.
    It separates the dataset into input features (X) and the target variable (y) using numpy slicing.
    It prints the shapes of X and y to give an idea of the dataset's size.
    It splits the dataset into training and test sets using the train_test_split function. 70% of the data is used for training and 30% for testing.
    Finally, it prints the shapes of the training and test sets to confirm the split.


In [ ]:
# load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
# load the dataset
df = read_csv('pima-indians-diabetes.csv', header=None)
# retrieve the array
data = df.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train (70%) and test sets (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Following from last step above (splitting the dataset into training and test sets using the train_test_split function), here are the additional steps:
  
    It initializes a Linear Regression model and fits it on the training data using the fit method.
    It makes predictions on the test data using the predict method.
    It calculates the MAE between the actual and predicted values using the mean_absolute_error function. MAE is a measure of prediction error that calculates the average absolute difference between actual and predicted values.
    It prints the MAE. The %.3f is a placeholder for a floating-point number, with 3 digits after the decimal point. The % operator is then used to insert the MAE into this placeholder.


In [ ]:
# evaluate model on the raw dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# load the dataset
df = read_csv('pima-indians-diabetes.csv', header=None)
# retrieve the array
data = df.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions using mean absolute error
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

Next, we can try removing outliers from the training dataset. The expectation is that the
outliers are causing the linear regression model to learn a bias or skewed understanding of the
problem, and that removing these outliers from the training set will allow a more e ective model
to be learned.

We can achieve this by defining the **LocalOutlierFactor** model and using it to
make a prediction on the training dataset, marking each row in the training dataset as normal
(1) or an outlier (-1). We will use the default hyperparameters for the outlier detection model,
although it is a good idea to tune the configuration to the specifics of your dataset.

In [ ]:
# evaluate model on training dataset with outliers removed
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
# load the dataset
df = read_csv('pima-indians-diabetes.csv', header=None)
# retrieve the array
data = df.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model without outliers
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

We can see MAE (Mean Absolute Error) reduced from to 0.324 0.317.